In [1]:
from RNN import MDNRNN
import torch
import numpy as np 
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_



cpu


In [2]:
################
#  DATALOADING #
################

In [2]:

def sliding_windows(data,seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[(i+1):(i+1)+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)



In [20]:
bsz = 200

data = np.load('latent.npy')
print(data[:,231233,:])
#Reshaping data and seperating training and test set
sc = MinMaxScaler()
data = sc.fit_transform(data.squeeze(axis = 0))

seq_length = 4
x, y = sliding_windows(data, seq_length)



train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])).float())
trainY = Variable(torch.Tensor(np.array(y[0:train_size])).float())


testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])).float())
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])).float())


trainingset =[]
for i in range(len(trainY)):
    trainingset.append((trainX[i,:,:], trainY[i]))
    


trainloader = torch.utils.data.DataLoader(trainingset, batch_size=bsz, shuffle=False)





[[-0.5650112   0.21304411  0.61334103 -1.42041063  0.13022065 -0.29030734
   1.66702461 -1.63919473  0.          0.          0.          0.
   0.          0.          0.          0.         -0.08      ]]


In [4]:
num_epochs = 2000
learning_rate = 1e-5

input_size = 17
hidden_size = 256
num_layers = 1
seq_length = 4
num_classes = 1
gaussians = 5

mdnrnn = MDNRNN(z_size = input_size, n_hidden = hidden_size, n_gaussians = gaussians, n_layers = num_layers)


def detach(states):
    return [state.detach() for state in states]

In [29]:
optimizer = torch.optim.Adam(mdnrnn.parameters(), lr = learning_rate)

epochs = 500
random = torch.zeros(200,4,17)

for epoch in range(epochs):
    # Set initial hidden and cell states
    hidden = mdnrnn.init_hidden(bsz)

    # Get mini-batch inputs and targets
    total_loss = 0
    for i, (inputs, targets) in enumerate(trainloader):
        # Forward pass
        if inputs.size(0) != bsz:
            continue
        hidden = detach(hidden)
        (pi, mu, sigma), hidden = mdnrnn(inputs, hidden)

        loss = mdnrnn.criterion(targets, pi, mu, sigma)
        # Backward and optimize
        mdnrnn.zero_grad()
        total_loss += loss.item()

        loss.backward()
        clip_grad_norm_(mdnrnn.parameters(), 0.5)
        optimizer.step()

    if epoch % 2 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'
               .format(epoch, epochs, total_loss))


Epoch [0/500], Loss: -2140.5096
Epoch [2/500], Loss: -5574.4729
Epoch [4/500], Loss: -5312.7406
Epoch [6/500], Loss: -4025.8504
Epoch [8/500], Loss: -5515.5596
Epoch [10/500], Loss: -5420.7950
Epoch [12/500], Loss: -4676.5475
Epoch [14/500], Loss: -5814.7942
Epoch [16/500], Loss: -5142.1563
Epoch [18/500], Loss: -5665.4992
Epoch [20/500], Loss: -4307.4893
Epoch [22/500], Loss: nan
Epoch [24/500], Loss: nan
Epoch [26/500], Loss: nan
Epoch [28/500], Loss: nan
Epoch [30/500], Loss: nan
Epoch [32/500], Loss: nan


KeyboardInterrupt: 

In [16]:
mdnrnn.load_state_dict(torch.load('../Models/transition-regression', map_location='cpu'))

mdnrnn.eval()
size = train_size+test_size
zero = np.random.randint(testX.size(0))
print(zero)
one = np.random.randint(testX.size(1))
print(one)
x = testX[zero:zero+1, one:one+1, :]
y = testX[zero:zero+1, one+1:one+2, :]

hidden = mdnrnn.init_hidden(1)
(pi, mu, sigma), _ = mdnrnn(x, hidden)

y_preds = [torch.normal(mu, sigma)[:, :, i, :] for i in range(5)]

print(testX[12341,0:1,:])

data_predict = sc.inverse_transform(y_preds)
dataY_truth = sc.inverse_transform(x)
print(data_predict[0:5])
print("---------------------------------------")
print(data_truth[0:5])

82676
3
tensor([[0.0631, 0.4084, 0.2199, 0.6749, 0.3187, 0.2388, 0.9545, 0.5471, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1293]])


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
torch.save(mdnrnn.state_dict(), '../Models/transition-regression')

In [ ]:
import pickle
with open('sc_midprice', 'wb') as f:
    pickle.dump(sc_y, f)
    